<a href="https://colab.research.google.com/github/Lakshana3/Sparse-CNN-Accelerator/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OpenLane Colab

This Google Colab notebook will:
* Install OpenLane and its dependencies
* Run a simple design, namely a serial-parallel multiplier, through the flow
  and targeting the [open source sky130 PDK](https://github.com/google/skywater-pdk/)
  by Google and Skywater.

In [1]:
# @title Setup Nix {display-mode: "form"}
# @markdown <img src="https://raw.githubusercontent.com/NixOS/nixos-artwork/master/logo/nix-snowflake.svg" width="32"/>
# @markdown
# @markdown Nix is a package manager with an emphasis on reproducible builds,
# @markdown and it is the primary method for installing OpenLane 2.
# @markdown
# @markdown This step installs the Nix package manager and enables the
# @markdown experimental "flakes" feature.
# @markdown
# @markdown If you're not in a Colab, this just sets the environment variables.
# @markdown You will need to install Nix and enable flakes on your own following
# @markdown [this guide](https://openlane2.readthedocs.io/en/stable/getting_started/common/nix_installation/index.html).
import os
import sys
import shutil

os.environ["LOCALE_ARCHIVE"] = "/usr/lib/locale/locale-archive"

if "google.colab" in sys.modules:
    if shutil.which("nix-env") is None:
        !curl -L https://nixos.org/nix/install | bash -s -- --daemon --yes
        !echo "extra-experimental-features = nix-command flakes" >> /etc/nix/nix.conf
        !killall nix-daemon
else:
    if shutil.which("nix-env") is None:
        raise RuntimeError("Nix is not installed!")

os.environ["PATH"] = f"/nix/var/nix/profiles/default/bin/:{os.getenv('PATH')}"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4267  100  4267    0     0   7511      0 --:--:-- --:--:-- --:--:--  7511
downloading Nix 2.30.2 binary tarball for x86_64-linux from 'https://releases.nixos.org/nix/nix-2.30.2/nix-2.30.2-x86_64-linux.tar.xz' to '/tmp/nix-binary-tarball-unpack.p7x2NQkd6o'...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.2M  100 24.2M    0     0  95.7M      0 --:--:-- --:--:-- --:--:-- 95.5M
Note: a multi-user installation is possible. See https://nixos.org/manual/nix/stable/installation/installing-binary.html#multi-user-installation
Switching to the Multi-user Installer
Welcome to the Multi-User Nix Installation

This installation tool will set up 

In [2]:
# @title Get OpenLane {display-mode: "form"}
# @markdown Click the ▷ button to download and install OpenLane.
# @markdown
# @markdown This will install OpenLane's tool dependencies using Nix,
# @markdown and OpenLane itself using PIP.
# @markdown
# @markdown Note that `python3-tk` may need to be installed using your OS's
# @markdown package manager.
import os
import subprocess
import IPython

openlane_version = "version-2.1"  # @param {key:"OpenLane Version", type:"string"}

if openlane_version == "latest":
    openlane_version = "main"

pdk_root = "~/.volare"  # @param {key:"PDK Root", type:"string"}

pdk_root = os.path.expanduser(pdk_root)

pdk = "sky130"  # @param {key:"PDK (without the variant)", type:"string"}

openlane_ipynb_path = os.path.join(os.getcwd(), "openlane_ipynb")

display(IPython.display.HTML("<h3>Downloading OpenLane…</a>"))


TESTING_LOCALLY = False
!rm -rf {openlane_ipynb_path}
!mkdir -p {openlane_ipynb_path}
if TESTING_LOCALLY:
    !ln -s {os.getcwd()} {openlane_ipynb_path}
else:
    !curl -L "https://github.com/efabless/openlane2/tarball/{openlane_version}" | tar -xzC {openlane_ipynb_path} --strip-components 1

try:
    import tkinter
except ImportError:
    if "google.colab" in sys.modules:
        !sudo apt-get install python-tk

try:
    import tkinter
except ImportError as e:
    display(
        IPython.display.HTML(
            '<h3 style="color: #800020";>❌ Failed to import the <code>tkinter</code> library for Python, which is required to load PDK configuration values. Make sure <code>python3-tk</code> or equivalent is installed on your system.</a>'
        )
    )
    raise e from None


display(IPython.display.HTML("<h3>Downloading OpenLane's dependencies…</a>"))
try:
    subprocess.check_call(
        ["nix", "profile", "install", ".#colab-env", "--accept-flake-config"],
        cwd=openlane_ipynb_path,
    )
except subprocess.CalledProcessError as e:
    display(
        IPython.display.HTML(
            '<h3 style="color: #800020";>❌ Failed to install binary dependencies using Nix…</h3>'
        )
    )

display(IPython.display.HTML("<h3>Downloading Python dependencies using PIP…</a>"))
try:
    subprocess.check_call(
        ["pip3", "install", "."],
        cwd=openlane_ipynb_path,
    )
except subprocess.CalledProcessError as e:
    display(
        IPython.display.HTML(
            '<h3 style="color: #800020";>❌ Failed to install Python dependencies using PIP…</h3>'
        )
    )
    raise e from None

display(IPython.display.HTML("<h3>Downloading PDK…</a>"))
import volare

volare.enable(
    volare.get_volare_home(pdk_root),
    pdk,
    open(
        os.path.join(openlane_ipynb_path, "openlane", "open_pdks_rev"),
        encoding="utf8",
    )
    .read()
    .strip(),
)

sys.path.insert(0, openlane_ipynb_path)
display(IPython.display.HTML("<h3>⭕️ Done.</a>"))

import logging

# Remove the stupid default colab logging handler
logging.getLogger().handlers.clear()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9491k    0 9491k    0     0  9827k      0 --:--:-- --:--:-- --:--:-- 16.2M


Version bdc9412b3e468c102d01b7cf6337be06ec6e9c9a not found locally, attempting to download…

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Version bdc9412b3e468c102d01b7cf6337be06ec6e9c9a enabled for the sky130 PDK.

In [13]:
import openlane

print(openlane.__version__)

2.1.11


### Creating the design

Now that OpenLane is set up, we can write a Verilog file as follows:

In [14]:
// ---------sparse_cnn_accelerator_single------//
module sparse_cnn_accelerator_single (
  input wire clk,
  input wire reset_async,
  input wire input_valid,
  input  wire [7:0]   shared_mask,
  input  wire [47:0]  shared_booth_enc,
  input wire [63:0] activations,
  output wire [17:0] result,
  output wire output_valid
);
  // Reset Synchronization
  reg rst_meta, rst_sync;
  always @(posedge clk or posedge reset_async) begin
    if (reset_async) {rst_meta,rst_sync} <= 2'b11;
    else             {rst_meta,rst_sync} <= {1'b0, rst_meta};
  end
  wire reset = rst_sync;

  // input Latching (on valid strobe)
  reg [63:0] act_reg;
  reg        stage1_valid;
  always @(posedge clk) begin
    if (reset) begin
      act_reg       <= 0;
      stage1_valid  <= 0;
    end else begin
      stage1_valid  <= input_valid;
      if (input_valid) begin
        act_reg       <= activations;
      end
    end
  end

  // MODIFIED: Uses shared mask instead of extracting from weight_mask
  wire [3:0] pop_count = shared_mask[0] + shared_mask[1] + shared_mask[2] + shared_mask[3] +
                        shared_mask[4] + shared_mask[5] + shared_mask[6] + shared_mask[7];
  wire sparsity_ge_50 = (pop_count <= 4);

  // Activation Selector (pipelined, robust: see below)
  wire [31:0] selected_activations;
  wire        selector_done;
  activation_selector act_sel (
    .clk(clk),
    .reset(reset),
    .mask(shared_mask), // MODIFIED: Uses shared mask
    .activations(act_reg),
    .selected_act(selected_activations),
    .done(selector_done)
  );

  // Valid Pipeline (for output valid alignment)
  // seven stage valid pipeline:
  reg [5:0] vpipe;
  always @(posedge clk) begin
    if (reset) vpipe <= 0;
    else       vpipe <= {vpipe[4:0], stage1_valid};
  end

  // MODIFIED: Block MAC with registered output
  wire [17:0] mac_result;
  wire        mac_valid;
  block_mac_unit blk_mac (
    .clk(clk),
    .reset(reset),
    .mask(shared_mask), // MODIFIED: Uses shared mask
    .booth_encoded(shared_booth_enc), // MODIFIED: Uses shared encoding
    .selected_activations(selected_activations),
    .selector_done(selector_done),
    .sparsity_ge_50(sparsity_ge_50),
    .result(mac_result),
    .valid(mac_valid)
  );

  // 8. Output Registering and Assignments
  reg [17:0] result_reg;
  always @(posedge clk) begin
    if (reset) begin
      result_reg      <= 0;
    end else if(vpipe[5]) begin
      // Register the MAC result
      result_reg      <= mac_result;
    end
  end
  assign result       = result_reg;
  assign output_valid = vpipe[5];

endmodule

// ========== Booth Encoder (pipelined output) ==========
module booth_pre_encoder (
  input  wire        clk,
  input  wire        reset,
  input  wire [31:0] weight,
  output reg  [47:0] booth_encoded
);
  function [2:0] booth_encode(input [2:0] bits);
    case (bits)
      3'b000: booth_encode = 3'b100; // 0
      3'b001: booth_encode = 3'b001; // +1
      3'b010: booth_encode = 3'b001; // +1
      3'b011: booth_encode = 3'b000; // +2
      3'b100: booth_encode = 3'b010; // -2
      3'b101: booth_encode = 3'b011; // -1
      3'b110: booth_encode = 3'b011; // -1
      3'b111: booth_encode = 3'b100; // 0
    endcase
  endfunction

  always @(posedge clk) begin
    if (reset) booth_encoded <= 48'd0;
    else begin
      booth_encoded[2:0]    <= booth_encode({weight[1:0], 1'b0});
      booth_encoded[5:3]    <= booth_encode(weight[3:1]);
      booth_encoded[8:6]    <= booth_encode(weight[5:3]);
      booth_encoded[11:9]   <= booth_encode(weight[7:5]);
      booth_encoded[14:12]  <= booth_encode({weight[9:8], 1'b0});
      booth_encoded[17:15]  <= booth_encode(weight[11:9]);
      booth_encoded[20:18]  <= booth_encode(weight[13:11]);
      booth_encoded[23:21]  <= booth_encode(weight[15:13]);
      booth_encoded[26:24]  <= booth_encode({weight[17:16], 1'b0});
      booth_encoded[29:27]  <= booth_encode(weight[19:17]);
      booth_encoded[32:30]  <= booth_encode(weight[21:19]);
      booth_encoded[35:33]  <= booth_encode(weight[23:21]);
      booth_encoded[38:36]  <= booth_encode({weight[25:24], 1'b0});
      booth_encoded[41:39]  <= booth_encode(weight[27:25]);
      booth_encoded[44:42]  <= booth_encode(weight[29:27]);
      booth_encoded[47:45]  <= booth_encode(weight[31:29]);
    end
  end
endmodule

// ========== MAC Unit with Partial Product Generator ==========
module generate_pprow (
  input  wire [2:0] ctrl,
  input  wire [7:0] x,
  output reg  [8:0] pp
);
  wire signed [8:0] sx = {x[7], x}; // Sign-extend

  always @(*) begin
    case (ctrl)
      3'b100: pp = 9'd0;         // 0
      3'b001: pp = sx;           // +1
      3'b000: pp = sx << 1;      // +2
      3'b010: pp = -(sx << 1);   // -2
      3'b011: pp = -sx;          // -1
      default: pp = 9'd0;
    endcase
  end
endmodule

module mac_unit (
    input  wire        clk,
    input  wire        reset,
    input  wire [11:0] enc,       // 4x Radix-4 Booth encodings (3 bits each)
    input  wire [7:0]  act,       // 8-bit activation
    output wire signed [16:0] result  // 17-bit output (for 18-bit accumulation)
);
    // ================================================
    // Clock Gating Logic (Booth-Zero Aware)
    // ================================================
    // MODIFIED: Correct Booth-zero detection
    function automatic is_booth_active(input [2:0] booth);
        return (booth != 3'b100);  // '100' = zero in Radix-4 Booth
    endfunction

    wire mac_clk_en = is_booth_active(enc[2:0])   ||  // Lane 0
                      is_booth_active(enc[5:3])   ||  // Lane 1
                      is_booth_active(enc[8:6])   ||  // Lane 2
                      is_booth_active(enc[11:9]);     // Lane 3

    (* gclk_enable = "mac_clk_en" *)
    wire gated_clk;
    assign gated_clk = clk & mac_clk_en;

    // MODIFIED: Partial Product Generation with gated clock
    wire [8:0] pp0, pp1, pp2, pp3;
    generate_pprow pp0_gen(.ctrl(enc[2:0]), .x(act), .pp(pp0));
    generate_pprow pp1_gen(.ctrl(enc[5:3]), .x(act), .pp(pp1));
    generate_pprow pp2_gen(.ctrl(enc[8:6]), .x(act), .pp(pp2));
    generate_pprow pp3_gen(.ctrl(enc[11:9]), .x(act), .pp(pp3));

    // MODIFIED: Wallace Tree Addition with registered output
    wire signed [16:0] sp0 = $signed(pp0);
    wire signed [16:0] sp1 = $signed(pp1) <<< 2;
    wire signed [16:0] sp2 = $signed(pp2) <<< 4;
    wire signed [16:0] sp3 = $signed(pp3) <<< 6;

    // Stage 1: CSA
    wire [16:0] s1 = sp0 ^ sp1 ^ sp2;
    wire [16:0] c1 = ((sp0 & sp1) | (sp0 & sp2) | (sp1 & sp2)) << 1;

    // Stage 2: CSA + Final CPA
    wire [16:0] s2 = s1 ^ c1 ^ sp3;
    wire [16:0] c2 = ((s1 & c1) | (s1 & sp3) | (c1 & sp3)) << 1;

    // MODIFIED: Register outpck
    reg signed [16:0] result_reg;
  always @(posedge gated_clk) begin // MAC result registers (clock-gated domain) sync reset
        if (reset) begin
            result_reg <= 0;
        end else begin
            result_reg <= s2 + c2;  // Final CPA
        end
    end

    assign result = result_reg;
endmodule

// ========== Activation Selector ==========
module activation_selector (
  input  wire        clk,
  input  wire        reset,
  input  wire [7:0]  mask,
  input  wire [63:0] activations,
  output reg  [31:0] selected_act,
  output reg         done
);
  //reg [7:0] mask_reg;
  reg [63:0] act_reg;
  always @(posedge clk) begin
    if (reset) begin
      //mask_reg <= 0;
      act_reg  <= 0;
    end else begin
      //mask_reg <= mask;
      act_reg  <= activations;
    end
  end

  function [2:0] pe(input [7:0] m); // priority encoder
    casez (m)
      8'b???????1: pe = 3'd0;
      8'b??????10: pe = 3'd1;
      8'b?????100: pe = 3'd2;
      8'b????1000: pe = 3'd3;
      8'b???10000: pe = 3'd4;
      8'b??100000: pe = 3'd5;
      8'b?1000000: pe = 3'd6;
      8'b10000000: pe = 3'd7;
      default: pe = 3'd0;
    endcase
  endfunction

  reg        cycle;
  reg [7:0]  rem_mask;

  //wire [3:0] popcount = mask_reg[0]+mask_reg[1]+mask_reg[2]+mask_reg[3]+mask_reg[4]+mask_reg[5]+mask_reg[6]+mask_reg[7];
  wire [3:0] popcount = mask[0]+mask[1]+mask[2]+mask[3]+mask[4]+mask[5]+mask[6]+mask[7];
  wire       sparse   = (popcount <= 4);

  //wire [7:0] sel_mask = (sparse || !cycle) ? mask_reg : rem_mask;.
  wire [7:0] sel_mask = (sparse || !cycle) ? mask : rem_mask;

  wire [2:0] i0 = pe(sel_mask);
  wire [7:0] m1 = sel_mask & ~(8'd1 << i0);
  wire [2:0] i1 = pe(m1);
  wire [7:0] m2 = m1      & ~(8'd1 << i1);
  wire [2:0] i2 = pe(m2);
  wire [7:0] m3 = m2      & ~(8'd1 << i2);
  wire [2:0] i3 = pe(m3);

  wire [7:0] a0 = act_reg[i0*8 +:8];
  wire [7:0] a1 = act_reg[i1*8 +:8];
  wire [7:0] a2 = act_reg[i2*8 +:8];
  wire [7:0] a3 = act_reg[i3*8 +:8];

  always @(posedge clk) begin
    if (reset) begin
      selected_act <= 0;
      done         <= 0;
      cycle        <= 0;
      rem_mask     <= 0;
    end else if (sparse) begin
      // Single cycle operation: output selection
      selected_act <= {a3,a2,a1,a0};
      done         <= 1;
      cycle        <= 0;
    end else if (!cycle) begin // first half dense
      selected_act <= {a3,a2,a1,a0};
      rem_mask     <= m3 & ~(8'd1 << i3);
      done         <= 1;
      cycle        <= 1;
    end else begin // second half dense
      selected_act <= {a3,a2,a1,a0};
      done         <= 1;
      cycle        <= 0;
    end
  end
endmodule

// ========== Block MAC Unit ==========
// Always update inputs on selector_done; only asserts valid after the second cycle if low sparsity
module block_mac_unit (
  input  wire        clk,
  input  wire        reset,
  input  wire [7:0]  mask,
  input  wire [47:0] booth_encoded,
  input  wire [31:0] selected_activations,
  input  wire        selector_done,
  input  wire        sparsity_ge_50,
  output reg  [17:0] result,
  output reg         valid
);
  // Input pipelining (latch on selector_done)
  reg [47:0] booth_encoded_r;
  reg [31:0] selected_activations_r;
  reg        selector_done_r;

  always @(posedge clk) begin
    if (reset) begin
      booth_encoded_r        <= 48'd0;
      selected_activations_r <= 32'd0;
      selector_done_r        <= 1'b0;
    end else begin
      selector_done_r <= selector_done;
      if (selector_done) begin
        booth_encoded_r        <= booth_encoded;
        selected_activations_r <= selected_activations;
      end
    end
  end

  // MACs for 4 lanes
  wire signed [16:0] mac0_result, mac1_result, mac2_result, mac3_result;
  mac_unit mac0 (.clk(clk), .reset(reset), .enc(booth_encoded[11:0]), .act(selected_activations[7:0]), .result(mac0_result));
  mac_unit mac1 (.clk(clk), .reset(reset), .enc(booth_encoded[23:12]), .act(selected_activations[15:8]), .result(mac1_result));
  mac_unit mac2 (.clk(clk), .reset(reset), .enc(booth_encoded[35:24]), .act(selected_activations[23:16]), .result(mac2_result));
  mac_unit mac3 (.clk(clk), .reset(reset), .enc(booth_encoded[47:36]), .act(selected_activations[31:24]), .result(mac3_result));

  wire signed [17:0] mac_sum =
  {mac0_result[16], mac0_result} +
  {mac1_result[16], mac1_result} +
  {mac2_result[16], mac2_result} +
  {mac3_result[16], mac3_result};

  // Dense-mode accumulator
  reg [17:0] partial_sum;
  reg accumulation_cycle;

  always @(posedge clk) begin
    if (reset) begin
      result <= 0;
      valid <= 0;
      partial_sum <= 0;
      accumulation_cycle <= 0;
    end else begin
      valid <= 0;         // Default: valid low
      if (selector_done_r) begin
        if (sparsity_ge_50) begin
          result <= mac_sum;
          valid <= 1;
        end else begin
          if (!accumulation_cycle) begin
            // First cycle of accumulation
            partial_sum <= mac_sum;
            accumulation_cycle <= 1;
          end else begin
            result <= partial_sum + mac_sum;
            valid <= 1;
            accumulation_cycle <= 0;
          end
        end
      end
    end
  end
endmodule

SyntaxError: unterminated string literal (detected at line 15) (ipython-input-3897707768.py, line 15)

### Setting up the configuration

OpenLane requries you to configure any Flow before using it. This is done using
the `config` module.

For colaboratories, REPLs and other interactive environments where there is no
concrete Flow object, the Configuration may be initialized using `Config.interactive`,
which will automatically propagate the configuration to any future steps.

You can find the documentation for `Config.interactive` [here](https://openlane2.readthedocs.io/en/latest/reference/api/config/index.html#openlane.config.Config.interactive).



In [15]:
from openlane.config import Config

Config.interactive(
    "sparse_cnn_accelerator_single",                           # <-- Your top-level module name
    PDK="sky130A",
    CLOCK_PORT="clk",
    CLOCK_NET="clk",
    CLOCK_PERIOD=10,                        # <-- ns
    #SDC="src/design.sdc",                   # <-- Path to your SDC, if available
    PRIMARY_GDSII_STREAMOUT_TOOL="klayout",
    # Optionally, set area hint (first run: let tool decide, later you can add DIE_AREA/CORE_AREA based on reports)
    # DIE_AREA=[0, 0, 500, 500],            # <-- Uncomment and adjust after first successful run, if you like
    # CORE_AREA=[50, 50, 450, 450],         # <-- As above
    # Any extra config options: e.g., timing signoff corners, lib/lef paths, etc.
)



### Interactive Configuration
#### Initial Values

<br />

```yaml
CELL_BB_VERILOG_MODELS:
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/verilog/sky130_fd_sc_hd__blackbox.v
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/verilog/sky130_fd_sc_hd__blackbox_pp.v
CELL_GDS:
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/gds/sky130_fd_sc_hd.gds
CELL_LEFS:
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/lef/sky130_fd_sc_hd.lef
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/lef/sky130_ef_sc_hd.lef
CELL_PAD_EXCLUDE:
- sky130_fd_sc_hd__tap*
- sky130_fd_sc_hd__decap*
- sky130_ef_sc_hd__decap*
- sky130_fd_sc_hd__fill*
CELL_SPICE_MODELS:
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/spice/sky130_ef_sc_hd__decap_12.spice
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/spice/sky130_ef_sc_hd__fill_12.spice
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/spice/sky130_ef_sc_hd__fill_4.spice
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/spice/sky130_ef_sc_hd__fill_8.spice
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/spice/sky130_fd_sc_hd.spice
CELL_VERILOG_MODELS:
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/verilog/primitives.v
- /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/verilog/sky130_fd_sc_hd.v
CLOCK_NET: clk
CLOCK_PERIOD: 10
CLOCK_PORT: clk
CLOCK_TRANSITION_CONSTRAINT: 0.15
CLOCK_UNCERTAINTY_CONSTRAINT: 0.25
CLOCK_WIRE_RC_LAYERS: null
DECAP_CELL:
- sky130_ef_sc_hd__decap_12
- sky130_fd_sc_hd__decap_8
- sky130_fd_sc_hd__decap_6
- sky130_fd_sc_hd__decap_4
- sky130_fd_sc_hd__decap_3
DEFAULT_CORNER: nom_tt_025C_1v80
DEFAULT_MAX_TRAN: null
DESIGN_DIR: .
DESIGN_NAME: sparse_cnn_accelerator_single
DIE_AREA: null
DIODE_CELL: sky130_fd_sc_hd__diode_2/DIODE
ENDCAP_CELL: sky130_fd_sc_hd__decap_3
EXTRA_EXCLUDED_CELLS: null
EXTRA_GDS_FILES: null
EXTRA_LEFS: null
EXTRA_LIBS: null
EXTRA_SPICE_MODELS: null
EXTRA_VERILOG_MODELS: null
FALLBACK_SDC_FILE: /usr/local/lib/python3.11/dist-packages/openlane/scripts/base.sdc
FILL_CELL:
- sky130_fd_sc_hd__fill*
FP_IO_HLAYER: met3
FP_IO_VLAYER: met2
FP_TAPCELL_DIST: 13
FP_TRACKS_INFO: /root/.volare/sky130A/libs.tech/openlane/sky130_fd_sc_hd/tracks.info
GND_NETS: null
GND_PIN: VGND
GPIO_PADS_LEF:
- /root/.volare/sky130A/libs.ref/sky130_fd_io/lef/sky130_fd_io.lef
- /root/.volare/sky130A/libs.ref/sky130_fd_io/lef/sky130_ef_io.lef
GPIO_PADS_LEF_CORE_SIDE:
- /root/.volare/sky130A/libs.tech/openlane/custom_cells/lef/sky130_fd_io_core.lef
- /root/.volare/sky130A/libs.tech/openlane/custom_cells/lef/sky130_ef_io_core.lef
GPIO_PADS_VERILOG:
- /root/.volare/sky130A/libs.ref/sky130_fd_io/verilog/sky130_ef_io.v
GPIO_PAD_CELLS:
- sky130_fd_io*
- sky130_ef_io*
IO_DELAY_CONSTRAINT: 20
LIB:
  '*_ff_n40C_1v95':
  - /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/lib/sky130_fd_sc_hd__ff_n40C_1v95.lib
  '*_ss_100C_1v60':
  - /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/lib/sky130_fd_sc_hd__ss_100C_1v60.lib
  '*_tt_025C_1v80':
  - /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/lib/sky130_fd_sc_hd__tt_025C_1v80.lib
MACROS: null
MAX_CAPACITANCE_CONSTRAINT: 0.2
MAX_FANOUT_CONSTRAINT: 10
MAX_TRANSITION_CONSTRAINT: 0.75
OUTPUT_CAP_LOAD: 33.442
PDK: sky130A
PDK_ROOT: /root/.volare
PLACE_SITE: unithd
PNR_EXCLUDED_CELL_FILE: /root/.volare/sky130A/libs.tech/openlane/sky130_fd_sc_hd/drc_exclude.cells
PRIMARY_GDSII_STREAMOUT_TOOL: klayout
RT_MAX_LAYER: met5
RT_MIN_LAYER: met1
SCL_GROUND_PINS:
- VGND
- VNB
SCL_POWER_PINS:
- VPWR
- VPB
SIGNAL_WIRE_RC_LAYERS: null
STA_CORNERS:
- nom_tt_025C_1v80
- nom_ss_100C_1v60
- nom_ff_n40C_1v95
- min_tt_025C_1v80
- min_ss_100C_1v60
- min_ff_n40C_1v95
- max_tt_025C_1v80
- max_ss_100C_1v60
- max_ff_n40C_1v95
STD_CELL_LIBRARY: sky130_fd_sc_hd
SYNTH_BUFFER_CELL: sky130_fd_sc_hd__buf_2/A/X
SYNTH_CLK_DRIVING_CELL: null
SYNTH_DRIVING_CELL: sky130_fd_sc_hd__inv_2/Y
SYNTH_EXCLUDED_CELL_FILE: /root/.volare/sky130A/libs.tech/openlane/sky130_fd_sc_hd/no_synth.cells
SYNTH_TIEHI_CELL: sky130_fd_sc_hd__conb_1/HI
SYNTH_TIELO_CELL: sky130_fd_sc_hd__conb_1/LO
TECH_LEFS:
  max_*: /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/techlef/sky130_fd_sc_hd__max.tlef
  min_*: /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/techlef/sky130_fd_sc_hd__min.tlef
  nom_*: /root/.volare/sky130A/libs.ref/sky130_fd_sc_hd/techlef/sky130_fd_sc_hd__nom.tlef
TIME_DERATING_CONSTRAINT: 5
TRISTATE_CELLS:
- sky130_fd_sc_hd__ebuf*
VDD_NETS: null
VDD_PIN: VPWR
VDD_PIN_VOLTAGE: 1.8
WELLTAP_CELL: sky130_fd_sc_hd__tapvpwrvgnd_1
WIRE_LENGTH_THRESHOLD: null
meta:
  flow: null
  openlane_version: 2.1.11
  step: null
  substituting_steps: null
  version: 1

```


### Running implementation steps

There are two ways to obtain OpenLane's built-in implementation steps:

* via directly importing from the `steps` module using its category:
    * `from openlane.steps import Yosys` then `Synthesis = Yosys.Synthesis`
* by using the step's id from the registry:
    * `from openlane.steps import Step` then `Synthesis = Step.factory.get("Yosys.Synthesis")`

You can find a full list of included steps here: https://openlane2.readthedocs.io/en/latest/reference/step_config_vars.html

In [16]:
from openlane.steps import Step

* First, get the step (and display its help)...

In [17]:
Synthesis = Step.factory.get("Yosys.Synthesis")

Synthesis.display_help()


(step-yosys-synthesis)=
### Synthesis

```{eval-rst}

Performs synthesis and technology mapping on Verilog RTL files
using Yosys and ABC, emitting a netlist.

Some metrics will also be extracted and updated, namely:

* ``design__instance__count``
* ``design__instance_unmapped__count``
* ``design__instance__area``

```

#### Importing
```python
from openlane.steps.yosys import Synthesis

# or

from openlane.steps import Step

Synthesis = Step.factory.get("Yosys.Synthesis")
```


#### Inputs and Outputs

| Inputs | Outputs |
| - | - |
|  | Verilog Netlist (.nl.v) |

(yosys.synthesis-configuration-variables)=
#### Configuration Variables

| Variable Name | Type | Description | Default | Units |
| - | - | - | - | - |
| `SYNTH_LATCH_MAP`{#var-yosys-synthesis-synth_latch_map}<sup>PDK</sup> | Path? | A path to a file containing the latch mapping for Yosys. | `None` |  |
| `SYNTH_TRISTATE_MAP`{#var-yosys-synthesis-synth_tristate_map}<sup>PDK</sup> | Path? | A path to a file containing the tri-state buffer mapping for Yosys. | `None` |  |
| `SYNTH_CSA_MAP`{#var-yosys-synthesis-synth_csa_map}<sup>PDK</sup> | Path? | A path to a file containing the carry-select adder mapping for Yosys. | `None` |  |
| `SYNTH_RCA_MAP`{#var-yosys-synthesis-synth_rca_map}<sup>PDK</sup> | Path? | A path to a file containing the ripple-carry adder mapping for Yosys. | `None` |  |
| `SYNTH_FA_MAP`{#var-yosys-synthesis-synth_fa_map}<sup>PDK</sup> | Path? | A path to a file containing the full adder mapping for Yosys. | `None` |  |
| `SYNTH_MUX_MAP`{#var-yosys-synthesis-synth_mux_map}<sup>PDK</sup> | Path? | A path to a file containing the mux mapping for Yosys. | `None` |  |
| `SYNTH_MUX4_MAP`{#var-yosys-synthesis-synth_mux4_map}<sup>PDK</sup> | Path? | A path to a file containing the mux4 mapping for Yosys. | `None` |  |
| `USE_LIGHTER`{#var-yosys-synthesis-use_lighter} | bool | Activates Lighter, an experimental plugin that attempts to optimize clock-gated flip-flops. | `False` |  |
| `LIGHTER_DFF_MAP`{#var-yosys-synthesis-lighter_dff_map} | Path? | An override to the custom DFF map file provided for the given SCL by Lighter. | `None` |  |
| `YOSYS_LOG_LEVEL`{#var-yosys-synthesis-yosys_log_level} | 'ALL'｜<br />'WARNING'｜<br />'ERROR' | Which log level for Yosys. At WARNING or higher, the initialization splash is also disabled. | `ALL` |  |
| `SYNTH_CHECKS_ALLOW_TRISTATE`{#var-yosys-synthesis-synth_checks_allow_tristate} | bool | Ignore multiple-driver warnings if they are connected to tri-state buffers on a best-effort basis. | `True` |  |
| `SYNTH_AUTONAME`{#var-yosys-synthesis-synth_autoname} | bool | Generates names for netlist instances. This results in instance names that can be extremely long, but are more human-readable. | `False` |  |
| `SYNTH_STRATEGY`{#var-yosys-synthesis-synth_strategy} | 'AREA 0'｜<br />'AREA 1'｜<br />'AREA 2'｜<br />'AREA 3'｜<br />'DELAY 0'｜<br />'DELAY 1'｜<br />'DELAY 2'｜<br />'DELAY 3'｜<br />'DELAY 4' | Strategies for abc logic synthesis and technology mapping. AREA strategies usually result in a more compact design, while DELAY strategies usually result in a design that runs at a higher frequency. Please note that there is no way to know which strategy is the best before trying them. | `AREA 0` |  |
| `SYNTH_ABC_BUFFERING`{#var-yosys-synthesis-synth_abc_buffering} | bool | Enables `abc` cell buffering. | `False` |  |
| `SYNTH_ABC_LEGACY_REFACTOR`{#var-yosys-synthesis-synth_abc_legacy_refactor} | bool | Replaces the ABC command `drf -l` with `refactor` which matches older versions of OpenLane but is more unstable. | `False` |  |
| `SYNTH_ABC_LEGACY_REWRITE`{#var-yosys-synthesis-synth_abc_legacy_rewrite} | bool | Replaces the ABC command `drw -l` with `rewrite` which matches older versions of OpenLane but is more unstable. | `False` |  |
| `SYNTH_DIRECT_WIRE_BUFFERING`{#var-yosys-synthesis-synth_direct_wire_buffering} | bool | Enables inserting buffer cells for directly connected wires. | `True` |  |
| `SYNTH_SPLITNETS`{#var-yosys-synthesis-synth_splitnets} | bool | Splits multi-bit nets into single-bit nets. Easier to trace but may not be supported by all tools. | `True` |  |
| `SYNTH_SIZING`{#var-yosys-synthesis-synth_sizing} | bool | Enables `abc` cell sizing (instead of buffering). | `False` |  |
| `SYNTH_NO_FLAT`{#var-yosys-synthesis-synth_no_flat} | bool | A flag that disables flattening the hierarchy during synthesis, only flattening it after synthesis, mapping and optimizations. | `False` |  |
| `SYNTH_SHARE_RESOURCES`{#var-yosys-synthesis-synth_share_resources} | bool | A flag that enables yosys to reduce the number of cells by determining shareable resources and merging them. | `True` |  |
| `SYNTH_ADDER_TYPE`{#var-yosys-synthesis-synth_adder_type} | 'YOSYS'｜<br />'FA'｜<br />'RCA'｜<br />'CSA' | Adder type to which the $add and $sub operators are mapped to.  Possible values are `YOSYS/FA/RCA/CSA`; where `YOSYS` refers to using Yosys internal adder definition, `FA` refers to full-adder structure, `RCA` refers to ripple carry adder structure, and `CSA` refers to carry select adder. | `YOSYS` |  |
| `SYNTH_EXTRA_MAPPING_FILE`{#var-yosys-synthesis-synth_extra_mapping_file} | Path? | Points to an extra techmap file for yosys that runs right after yosys `synth` before generic techmap. | `None` |  |
| `SYNTH_PARAMETERS`{#var-yosys-synthesis-synth_parameters} | List[str]? | Key-value pairs to be `chparam`ed in Yosys, in the format `key1=value1`. | `None` |  |
| `SYNTH_ELABORATE_ONLY`{#var-yosys-synthesis-synth_elaborate_only} | bool | "Elaborate" the design only without attempting any logic mapping. Useful when dealing with structural Verilog netlists. | `False` |  |
| `SYNTH_ELABORATE_FLATTEN`{#var-yosys-synthesis-synth_elaborate_flatten} | bool | If `SYNTH_ELABORATE_ONLY` is specified, this variable controls whether or not the top level should be flattened. | `True` |  |
| `VERILOG_FILES`{#var-yosys-synthesis-verilog_files} | List[Path] | The paths of the design's Verilog files. | `None` |  |
| `VERILOG_DEFINES`{#var-yosys-synthesis-verilog_defines} | List[str]? | Preprocessor defines for input Verilog files. | `None` |  |
| `VERILOG_POWER_DEFINE`{#var-yosys-synthesis-verilog_power_define} | str | Specifies the name of the define used to guard power and ground connections in the input RTL. | `USE_POWER_PINS` |  |
| `VERILOG_INCLUDE_DIRS`{#var-yosys-synthesis-verilog_include_dirs} | List[str]? | Specifies the Verilog `include` directories. | `None` |  |
| `USE_SYNLIG`{#var-yosys-synthesis-use_synlig} | bool | Use the Synlig plugin to process files, which has better SystemVerilog parsing capabilities but may not be compatible with all Yosys commands and attributes. | `False` |  |
| `SYNLIG_DEFER`{#var-yosys-synthesis-synlig_defer} | bool | Uses -defer flag when reading files the Synlig plugin, which may improve performance by reading each file separately, but is experimental. | `False` |  |



* Then run it. Note you can pass step-specific configs using Python keyword
  arguments.

### Synthesis

We need to start by converting our high-level Verilog to one that just shows
the connections between small silicon patterns called "standard cells" in process
called Synthesis. We can do this by passing the Verilog files as a configuration
variable to `Yosys.Synthesis` as follows, then running it.

As this is the first step, we need to create an empty state and pass it to it.

In [19]:
from openlane.state import State

synthesis = Synthesis(
    VERILOG_FILES=["./sparse_cnn_accelerator_single.v"],
    state_in=State(),
    USE_SYNLIG=True, # Use the SystemVerilog frontend for Yosys
)
synthesis.start()

InvalidConfig: The following errors were encountered: 
	* Path provided for variable 'VERILOG_FILES[0]' is invalid: './sparse_cnn_accelerator_single.v' does not exist

In [ ]:
display(synthesis)

### Floorplanning

Floorplanning does two things:

* Determines the dimensions of the final chip.
* Creates the "cell placement grid" which placed cells must be aligned to.
    * Each cell in the grid is called a "site." Cells can occupy multiple
      sites, with the overwhelming majority of cells occupying multiple sites
      by width, and some standard cell libraries supporting varying heights as well.

> Don't forget- you may call `display_help()` on any Step class to get a full
> list of configuration variables.


In [ ]:
Floorplan = Step.factory.get("OpenROAD.Floorplan")

floorplan = Floorplan(state_in=synthesis.state_out)
floorplan.start()

In [ ]:
display(floorplan)

### Tap/Endcap Cell Insertion

This places two kinds of cells on the floorplan:

* End cap/boundary cells: Added at the beginning and end of each row. True to
  their name, they "cap off" the core area of a design.
* Tap cells: Placed in a polka dot-ish fashion across the rows. Tap cells
  connect VDD to the nwell and the psubstrate to VSS, which the majority of cells
  do not do themselves to save area- but if you go long enough without one such
  connection you end up with the cell "latching-up"; i.e.; refusing to switch
  back to LO from HI.

  There is a maximum distance between tap cells enforced as part of every
  foundry process.

In [ ]:
TapEndcapInsertion = Step.factory.get("OpenROAD.TapEndcapInsertion")

tdi = TapEndcapInsertion(state_in=floorplan.state_out)
tdi.start()

In [ ]:
display(tdi)

### I/O Placement

This places metal pins at the edges of the design corresponding to the top level
inputs and outputs for your design. These pins act as the interface with other
designs when you integrate it with other designs.

In [ ]:
IOPlacement = Step.factory.get("OpenROAD.IOPlacement")

ioplace = IOPlacement(state_in=tdi.state_out)
ioplace.start()

In [ ]:
display(ioplace)

### Generating the Power Distribution Network (PDN)

This creates the power distribution network for your design, which is essentially
a plaid pattern of horizontal and vertical "straps" across the design that is
then connected to the rails' VDD and VSS (via the tap cells.)

You can find an explanation of how the power distribution network works at this
link: https://openlane2.readthedocs.io/en/latest/usage/hardening_macros.html#pdn-generation

While we typically don't need to mess with the PDN too much, the SPM is a small
design, so we're going to need to make the plaid pattern formed by the PDN a bit
smaller.

In [ ]:
GeneratePDN = Step.factory.get("OpenROAD.GeneratePDN")

pdn = GeneratePDN(
    state_in=ioplace.state_out,
    FP_PDN_VWIDTH=2,
    FP_PDN_HWIDTH=2,
    FP_PDN_VPITCH=30,
    FP_PDN_HPITCH=30,
)
pdn.start()

In [ ]:
display(pdn)

### Global Placement

Global Placement is deciding on a fuzzy, non-final location for each of the cells,
with the aim of minimizing the distance between cells that are connected
together (more specifically, the total length of the not-yet-created wires that
will connect them).

As you will see in the `.display()` in the second cell below, the placement is
considered "illegal", i.e., not properly aligned with the cell placement grid.
This is addressed by "Detailed Placement", also referred to as "placement
legalization", which is the next step.

In [ ]:
GlobalPlacement = Step.factory.get("OpenROAD.GlobalPlacement")

gpl = GlobalPlacement(state_in=pdn.state_out)
gpl.start()

In [ ]:
display(gpl)

### Detailed Placement

This aligns the fuzzy placement from before with the grid, "legalizing" it.

In [ ]:
DetailedPlacement = Step.factory.get("OpenROAD.DetailedPlacement")

dpl = DetailedPlacement(state_in=gpl.state_out)
dpl.start()

In [ ]:
display(dpl)

### Clock Tree Synthesis (CTS)

With the cells now having a final placement, we can go ahead and create what
is known as the clock tree, i.e., the hierarchical set of buffers used
for clock signal to minimize what is known as "clock skew"- variable delay
of the clock cycle from register to register because of factors such as metal
wire length, clock load (number of gates connected to the same clock buffer,)
et cetera.

The CTS step creates the cells and places the between the gaps in the detailed
placement above.

In [ ]:
CTS = Step.factory.get("OpenROAD.CTS")

cts = CTS(state_in=dpl.state_out)
cts.start()

In [ ]:
display(cts)

### Global Routing

Global routing "plans" the routes the wires between two gates (or gates and
I/O pins/the PDN) will take. The results of global routing (which are called
"routing guides") are stored in internal data structures and have no effect on
the actual design, so there is no `display()` statement.

In [ ]:
GlobalRouting = Step.factory.get("OpenROAD.GlobalRouting")

grt = GlobalRouting(state_in=cts.state_out)
grt.start()

### Detailed Routing

Detailed routing uses the guides from Global Routing to actually create wires
on the metal layers and connect the gates, making the connections finally physical.

This is typically the longest step in the flow.

In [ ]:
DetailedRouting = Step.factory.get("OpenROAD.DetailedRouting")

drt = DetailedRouting(state_in=grt.state_out)
drt.start()

In [ ]:
display(drt)

### Fill Insertion

Finally, as we're done placing all the essential cells, the only thing left to
do is fill in the gaps.

We prioritize the use of decap (decoupling capacitor) cells, which
further supports the power distribution network, but when there aren't any
small enough cells, we just use regular fill cells.

In [ ]:
FillInsertion = Step.factory.get("OpenROAD.FillInsertion")

fill = FillInsertion(state_in=drt.state_out)
fill.start()

In [ ]:
display(fill)

### Parasitics Extraction a.k.a. Resistance/Capacitance Extraction (RCX)

This step does not alter the design- rather, it computes the
[Parasitic elements](https://en.wikipedia.org/wiki/Parasitic_element_(electrical_networks))
of the circuit, which have an effect of timing, as we prepare to do the final
timing analysis.

The parasitic elements are saved in the **Standard Parasitics Exchange Format**,
or SPEF. OpenLane creates a SPEF file for each interconnect corner as described in
the [Corners and STA](https://openlane2.readthedocs.io/en/latest/usage/corners_and_sta.html)
section of the documentation.

In [ ]:
RCX = Step.factory.get("OpenROAD.RCX")

rcx = RCX(state_in=fill.state_out)
rcx.start()

### Static Timing Analysis (Post-PnR)

STA is a process that verifies that a chip meets certain constraints on clock
and data timings to run at its rated clock speed. See [Corners and STA](https://openlane2.readthedocs.io/en/latest/usage/corners_and_sta.html)
in the documentation for more info.

---

This step generates two kinds of files:
* `.lib`: Liberty™-compatible Library files. Can be used to do static timing
  analysis when creating a design with this design as a sub-macro.
* `.sdf`: Standard Delay Format. Can be used with certain simulation software
  to do *dynamic* timing analysis.

Unfortunately, the `.lib` files coming out of OpenLane right now are not super
reliable for timing purposes and are only provided for completeness.
When using OpenLane-created macros withing other designs, it is best to use the
macro's final netlist and extracted parasitics instead.

In [ ]:
STAPostPNR = Step.factory.get("OpenROAD.STAPostPNR")

sta_post_pnr = STAPostPNR(state_in=rcx.state_out)
sta_post_pnr.start()

### Stream-out

Stream-out is the process of converting the designs from the abstract formats
using during floorplanning, placement and routing into a concrete format called
GDSII (lit. Graphic Design System 2), which is the final file that is then sent
for fabrication.

In [ ]:
StreamOut = Step.factory.get("KLayout.StreamOut")

gds = StreamOut(state_in=sta_post_pnr.state_out)
gds.start()

In [ ]:
display(gds)

### Design Rule Checks (DRC)

DRC determines that the final layout does not violate any of the rules set by
the foundry to ensure the design is actually manufacturable- for example,
not enough space between two wires, *too much* space between tap cells, and so
on.

A design not passing DRC will typically be rejected by the foundry, who
also run DRC on their side.

In [ ]:
DRC = Step.factory.get("Magic.DRC")

drc = DRC(state_in=gds.state_out)
drc.start()

### SPICE Extraction for Layout vs. Schematic Check

This step tries to reconstruct a SPICE netlist from the GDSII file, so it can
later be used for the **Layout vs. Schematic** (LVS) check.

In [ ]:
SpiceExtraction = Step.factory.get("Magic.SpiceExtraction")

spx = SpiceExtraction(state_in=drc.state_out)
spx.start()

### Layout vs. Schematic (LVS)

A comparison between the final Verilog netlist (from PnR) and the final
SPICE netlist (extracted.)

This check effectively compares the physically implemented circuit to the final
Verilog netlist output by OpenROAD.

The idea is, if there are any disconnects, shorts or other mismatches in the
physical implementation that do not exist in the logical view of the design,
they would be caught at this step.

Common issues that result in LVS violations include:
* Lack of fill cells or tap cells in the design
* Two unrelated signals to be shorted, or a wire to be disconnected (most
  commonly seen with misconfigured PDN)

Chips with LVS errors are typically dead on arrival.

In [ ]:
LVS = Step.factory.get("Netgen.LVS")

lvs = LVS(state_in=spx.state_out)
lvs.start()